# Data Clean-up Assumptions

In developing cleaned versions of the Littlepay data tables for analysis. The cleaned tables that we've developed so far are for:
* `device_transactions`
* `micropayment_adjustments`
* `micropayment_device_transactions`
* `micropayments`
* `product_data`

In [4]:
from calitp import magics
import os
os.environ['AIRFLOW_ENV'] = 'development'

## Device Transactions

We're assuming that the `littlepay_device_transaction_id` field should be a unique key.

In [9]:
%%sql

with nonexact_dupe_ids as (
    select littlepay_transaction_id, count(distinct calitp_hash) as calitp_n_hashes
    from payments.stg_enriched_device_transactions
    group by 1
)

select *
from payments.stg_enriched_device_transactions
join nonexact_dupe_ids using (littlepay_transaction_id)
where calitp_n_hashes > 1
order by littlepay_transaction_id, calitp_export_datetime desc

,littlepay_transaction_id,participant_id,customer_id,device_transaction_id,device_id,device_id_issuer,type,transaction_outcome,transction_deny_reason,transaction_date_time_utc,...,calitp_extracted_at,calitp_hash,calitp_file_name,calitp_export_account,calitp_export_datetime,calitp_n_dupes,calitp_n_dupe_ids,calitp_dupe_number,transaction_date_time_pacific,calitp_n_hashes
0,0004dabe-abde-4539-8c60-ba49ff7b8c25,mst,2e8bfdc1-ce64-4630-a6ff-3035b8e6581f,36481BC5-7633-A21C-3EB6-484093BD91D3,17F1267C,Littlepay,single,allow,None,2021-07-22T22:21:48.000Z,...,2021-11-18,PuVmTG1NA7e+H4ne/rePwg==,gs://gtfs-data-test/payments-processed/device_...,mst,2021-07-24 06:43:00,1,2,1,2021-07-22 15:21:48,2
1,0004dabe-abde-4539-8c60-ba49ff7b8c25,mst,2e8bfdc1-ce64-4630-a6ff-3035b8e6581f,36481BC5-7633-A21C-3EB6-484093BD91D3,17F1267C,Littlepay,single,allow,None,2021-07-22T22:21:48.000Z,...,2021-11-18,SJu3E1EgYOb1FCul40j17w==,gs://gtfs-data-test/payments-processed/device_...,mst,2021-07-23 06:34:00,1,2,2,2021-07-22 15:21:48,2
2,0005d3ba-8f49-425e-bde4-2eb2c9b242a1,mst,b9563c3b-0fb6-405b-9c61-c0606cfad025,BD44C262-7ABA-B46B-0CCC-71167E04C481,17F12678,Littlepay,single,allow,None,2021-10-28T23:45:27.000Z,...,2021-11-18,7ZtsLVYZASJdnbg/9dLnrA==,gs://gtfs-data-test/payments-processed/device_...,mst,2021-10-30 04:55:00,1,2,1,2021-10-28 16:45:27,2
3,0005d3ba-8f49-425e-bde4-2eb2c9b242a1,mst,b9563c3b-0fb6-405b-9c61-c0606cfad025,BD44C262-7ABA-B46B-0CCC-71167E04C481,17F12678,Littlepay,single,allow,None,2021-10-28T23:45:27.000Z,...,2021-11-18,7KsjsUMibSUMZlsLQCc54Q==,gs://gtfs-data-test/payments-processed/device_...,mst,2021-10-29 04:56:00,1,2,2,2021-10-28 16:45:27,2
4,000df49a-aff8-422a-abbe-7c6b97123b6c,mst,e600de16-582c-4795-a433-7780adb4985d,C7741986-B015-55C4-B448-8DD5085D8499,17F12523,Littlepay,single,allow,None,2021-08-30T23:26:35.000Z,...,2021-11-18,FWCuHX6IdmX4SsQUZwp5dw==,gs://gtfs-data-test/payments-processed/device_...,mst,2021-09-01 05:39:00,1,2,1,2021-08-30 16:26:35,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5821,ffcdebb3-1dfd-4217-9d35-551fd6ad9dda,mst,948b18d4-a584-4179-96ba-f4db5f966af4,E9878665-BC84-61E2-574D-2D08ED6E00D8,17F12971,Littlepay,single,allow,None,2021-11-02T13:33:00.000Z,...,2021-11-18,4Jkhdg2ycxosYOGceYVWhQ==,gs://gtfs-data-test/payments-processed/device_...,mst,2021-11-03 04:57:00,1,2,2,2021-11-02 06:33:00,2
5822,ffd138b4-afcf-479a-a5bb-909d104f18de,mst,41b0b239-544d-4dbc-aeff-89d1c178275d,E9D13C33-B0DA-A553-C512-5D7384976009,17F12784,Littlepay,single,allow,None,2021-11-10T22:46:15.000Z,...,2021-11-18,5BGBYGpaiGbPMBBsILgUmA==,gs://gtfs-data-test/payments-processed/device_...,mst,2021-11-12 05:06:00,1,2,1,2021-11-10 14:46:15,2
5823,ffd138b4-afcf-479a-a5bb-909d104f18de,mst,41b0b239-544d-4dbc-aeff-89d1c178275d,E9D13C33-B0DA-A553-C512-5D7384976009,17F12784,Littlepay,single,allow,None,2021-11-10T22:46:15.000Z,...,2021-11-18,WdFUhMR56gznhO10nNRQSw==,gs://gtfs-data-test/payments-processed/device_...,mst,2021-11-11 05:11:00,1,2,2,2021-11-10 14:46:15,2
5824,fff49a28-ed1f-42ca-a6fe-be70f4ca99de,mst,e600de16-582c-4795-a433-7780adb4985d,7AED05DA-8D31-8D1B-E3B8-98BEB6CB8C10,17F12677,Littlepay,single,allow,None,2021-09-24T19:15:36.000Z,...,2021-11-18,tigDEdNrEFCkA4+dQADbvQ==,gs://gtfs-data-test/payments-processed/device_...,mst,2021-09-26 04:54:00,1,2,1,2021-09-24 12:15:36,2
